In [1]:
import os

In [2]:
os.chdir("../../")

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BikeModelEvalConfig:
    root_dir: Path
    X_test: Path
    y_test: Path
    tunned_model: Path
    mlflow_uri: str
    X_train: Path
    y_train: Path
    score: Path
    


In [5]:
from src.bike.constants import *
from src.bike.utils.common import read_yaml,create_directory

[2024-10-02 20:56:44,342: INFO]: Logging file start


In [6]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.data_root])

    def get_model_eval(self)-> BikeModelEvalConfig:
        config = self.config.model_eval

        create_directory([config.root_dir])

        model_eval = BikeModelEvalConfig(
            root_dir=config.root_dir,
            X_test=config.X_test,
            y_test=config.y_test,
            tunned_model=config.tunned_model,
            mlflow_uri="https://dagshub.com/Vicky7873/95Mobiles.mlflow",
            X_train=config.X_train,
            y_train=config.y_train,
            score= config.score
        )

        return model_eval

In [7]:
from sklearn.metrics import r2_score
import pandas as pd
import dagshub
import mlflow

In [8]:
import joblib
import pickle
import json

In [9]:
class BikeModelEval:
    def __init__(self,config:BikeModelEvalConfig):
        self.config = config

    def model_eval(self):
        dagshub.init(repo_owner='Vicky7873', repo_name='95Mobiles', mlflow=True)
        X_test = pd.read_csv(self.config.X_test)
        y_test = pd.read_csv(self.config.y_test)
        model = joblib.load(self.config.tunned_model)
        X_train = pd.read_csv(self.config.X_train)
        y_train = pd.read_csv(self.config.y_train)

        mlflow.set_registry_uri(uri=self.config.mlflow_uri)
        mlflow.set_experiment("Bike model evalution")

        with mlflow.start_run():
            self.y_pred_train = model.predict(X_train)
            self.y_pred = model.predict(X_test)

            self.y_train_acc = r2_score(y_train,self.y_pred_train)
            self.y_test_acc = r2_score(y_test,self.y_pred)
            print("test acc: ",r2_score(y_test,self.y_pred))
            print("train acc: ",r2_score(y_train,self.y_pred_train))
            mlflow.log_metric("testing_Acc", self.y_test_acc)
            mlflow.log_metric("training_Acc", self.y_train_acc)


    def save_score(self):
        scores = {
            "train acc " : self.y_train_acc,
            "test_acc ": self.y_test_acc
        }
        with open (self.config.score,'w') as file:
            json.dump(scores,file,indent=4)

In [10]:
try:
    config = ConfigurationManger()
    eval_config = config.get_model_eval()
    bike_model_config =BikeModelEval(config=eval_config)
    bike_model_config.model_eval()
    bike_model_config.save_score()
except Exception as e:
    raise e

[2024-10-02 20:56:46,006: INFO]: yaml: config/bike_config.yaml loaded successfully
[2024-10-02 20:56:46,009: INFO]: yaml: params/bike_params.yaml loaded successfully
[2024-10-02 20:56:46,009: INFO]: created directory at: data/bike
[2024-10-02 20:56:46,010: INFO]: created directory at: data/bike/model_eval
[2024-10-02 20:56:46,506: INFO]: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as Vicky7873

[2024-10-02 20:56:46,512: INFO]: Accessing as Vicky7873
[2024-10-02 20:56:47,016: INFO]: HTTP Request: GET https://dagshub.com/api/v1/repos/Vicky7873/95Mobiles "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Vicky7873/95Mobiles"

[2024-10-02 20:56:47,019: INFO]: Initialized MLflow to track repo "Vicky7873/95Mobiles"


Repository Vicky7873/95Mobiles initialized!

[2024-10-02 20:56:47,021: INFO]: Repository Vicky7873/95Mobiles initialized!
test acc:  0.8249236193415848
train acc:  0.8226296099542652


2024/10/02 20:56:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run respected-snake-211 at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/2/runs/d11f094ef277443bb24002ca5ed9f452.
2024/10/02 20:56:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/2.
